In [1]:
import requests
import json
from pprint import pprint
import numpy as np
import pandas as pd
import os
import datetime as dt

import sqlalchemy
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date
from opencage.geocoder import OpenCageGeocode

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session


from flask import Flask, jsonify

In [ ]:
results = requests.get('https://public.opendatasoft.com/api/records/1.0/download/?dataset=air-bnb-listings&format=json').json()


In [21]:
pprint(results.fields)

0      {'updated_date': '2019-05-05', 'reviews_per_mo...
1      {'updated_date': '2019-05-05', 'reviews_per_mo...
2      {'updated_date': '2019-05-05', 'reviews_per_mo...
3      {'updated_date': '2019-05-05', 'reviews_per_mo...
4      {'updated_date': '2019-05-05', 'reviews_per_mo...
5      {'updated_date': '2019-05-05', 'reviews_per_mo...
6      {'updated_date': '2019-05-05', 'reviews_per_mo...
7      {'updated_date': '2019-05-05', 'reviews_per_mo...
8      {'updated_date': '2019-05-05', 'reviews_per_mo...
9      {'updated_date': '2019-05-05', 'reviews_per_mo...
10     {'updated_date': '2019-05-05', 'reviews_per_mo...
11     {'updated_date': '2019-05-05', 'reviews_per_mo...
12     {'updated_date': '2019-05-05', 'reviews_per_mo...
13     {'updated_date': '2019-05-05', 'reviews_per_mo...
14     {'updated_date': '2019-05-05', 'reviews_per_mo...
15     {'updated_date': '2019-05-05', 'reviews_per_mo...
16     {'updated_date': '2019-05-05', 'reviews_per_mo...
17     {'updated_date': '2019-0

In [23]:
results_df = pd.read_json(results["fields"].to_json())
results_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
availability_365,347,302,206,0,25,335,146,0,42,244,...,356,365,360,315,314,0,281,249,338,75
calculated_host_listings_count,74,1,1,2,4,17,1,1,3,1,...,1,4,10,3,1,5,2,1,2,3
city,New-orleans,New-orleans,New-orleans,New-orleans,New-orleans,New-orleans,New-orleans,New-orleans,New-orleans,New-orleans,...,Rome,Rome,Rome,Rome,Rome,Rome,Rome,Rome,Rome,Rome
column_10,1003,74,175,86,84,296,73,130,250,200,...,39,89,350,20,48,75,119,60,79,25
column_19,United states,United states,United states,United states,United states,United states,United states,United states,United states,United states,...,Italy,Italy,Italy,Italy,Italy,Italy,Italy,Italy,Italy,Italy


In [26]:
transpose = results_df.transpose()
transpose.head()

,availability_365,calculated_host_listings_count,city,column_10,column_19,column_20,coordinates,host_id,id,last_review,minimum_nights,name,neighbourhood,number_of_reviews,reviews_per_month,room_type,updated_date
0,347,74,New-orleans,1003,United states,"United states, New-orleans, Central Business D...","[29.9528609234, -90.0713113928]",114353388,19051656,2017-10-19,1,Gorgeous Stay Alfred on North Carondelet Street,Central Business District,1,0.05,Entire home/apt,2019-05-05
1,302,1,New-orleans,74,United states,"United states, New-orleans, Viavant - Venetian...","[29.9870343373, -89.9449684809]",61310992,19079967,2019-04-28,1,Cozy New Orleans Efficiency On The Water,Viavant - Venetian Isles,20,4.84,Entire home/apt,2019-05-05
2,206,1,New-orleans,175,United states,"United states, New-orleans, Marlyville - Fonta...","[29.9547710049, -90.119942163]",22171971,19123547,2019-04-22,2,Townhome on the Streetcar Line,Marlyville - Fontainbleau,74,3.65,Entire home/apt,2019-05-05
3,0,2,New-orleans,86,United states,"United states, New-orleans, Lower Garden District","[29.9380891716, -90.0735907443]",710534,19245713,2019-04-29,2,Stylish 1BR Shotgun in Lower Garden District,Lower Garden District,89,3.9,Entire home/apt,2019-05-05
4,25,4,New-orleans,84,United states,"United states, New-orleans, Seventh Ward","[29.9718554828, -90.0692218041]",2428040,19267950,2019-04-27,1,Relaxing 3BR Balcony Apartment,Seventh Ward,22,1.08,Entire home/apt,2019-05-05


In [28]:
cleanData = transpose[[ 'column_10','id', 'coordinates', 'room_type', "city", 'column_19']]
cleanData.head()

,column_10,id,coordinates,room_type,city,column_19
0,1003,19051656,"[29.9528609234, -90.0713113928]",Entire home/apt,New-orleans,United states
1,74,19079967,"[29.9870343373, -89.9449684809]",Entire home/apt,New-orleans,United states
2,175,19123547,"[29.9547710049, -90.119942163]",Entire home/apt,New-orleans,United states
3,86,19245713,"[29.9380891716, -90.0735907443]",Entire home/apt,New-orleans,United states
4,84,19267950,"[29.9718554828, -90.0692218041]",Entire home/apt,New-orleans,United states


In [29]:
cleanData = cleanData.rename(index=str, columns={"column_10":"Price",
                                                 "id":"Room_Id", 
                                                 "coordinates":"Coordinates",
                                                 "column_19":"Country", 
                                                 "room_type":"Room_Type",
                                                 "city":"City"})

cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country
0,1003,19051656,"[29.9528609234, -90.0713113928]",Entire home/apt,New-orleans,United states
1,74,19079967,"[29.9870343373, -89.9449684809]",Entire home/apt,New-orleans,United states
2,175,19123547,"[29.9547710049, -90.119942163]",Entire home/apt,New-orleans,United states
3,86,19245713,"[29.9380891716, -90.0735907443]",Entire home/apt,New-orleans,United states
4,84,19267950,"[29.9718554828, -90.0692218041]",Entire home/apt,New-orleans,United states


In [40]:
# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cleanData['Coordinates']:
   # Try to,
   try:
       # Split the row by comma and append
       # everything before the comma to lat
       lat.append(row[0])
       # Split the row by comma and append
       # everything after the comma to lon
       lon.append(row[1])
   # But if you get an error
   except:
       # append a missing value to lat
       lat.append(np.NaN)
       # append a missing value to lon
       lon.append(np.NaN)

# Create two new columns from lat and lon
cleanData['Latitude'] = lat
cleanData['Longitude'] = lon



In [46]:
cleanData.drop(columns=["Coordinates"])
cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Latitude,Longitude
0,1003,19051656,"[29.9528609234, -90.0713113928]",Entire home/apt,New-orleans,United states,29.952861,-90.071311
1,74,19079967,"[29.9870343373, -89.9449684809]",Entire home/apt,New-orleans,United states,29.987034,-89.944968
2,175,19123547,"[29.9547710049, -90.119942163]",Entire home/apt,New-orleans,United states,29.954771,-90.119942
3,86,19245713,"[29.9380891716, -90.0735907443]",Entire home/apt,New-orleans,United states,29.938089,-90.073591
4,84,19267950,"[29.9718554828, -90.0692218041]",Entire home/apt,New-orleans,United states,29.971855,-90.069222


In [48]:
cleanData["Selection"] = cleanData["City"]+", "+cleanData["Country"]
cleanData.head()

,Price,Room_Id,Coordinates,Room_Type,City,Country,Latitude,Longitude,Selection
0,1003,19051656,"[29.9528609234, -90.0713113928]",Entire home/apt,New-orleans,United states,29.952861,-90.071311,"New-orleans, United states"
1,74,19079967,"[29.9870343373, -89.9449684809]",Entire home/apt,New-orleans,United states,29.987034,-89.944968,"New-orleans, United states"
2,175,19123547,"[29.9547710049, -90.119942163]",Entire home/apt,New-orleans,United states,29.954771,-90.119942,"New-orleans, United states"
3,86,19245713,"[29.9380891716, -90.0735907443]",Entire home/apt,New-orleans,United states,29.938089,-90.073591,"New-orleans, United states"
4,84,19267950,"[29.9718554828, -90.0692218041]",Entire home/apt,New-orleans,United states,29.971855,-90.069222,"New-orleans, United states"


In [ ]:
conn_str = "root:***password***@localhost/airbnb_data"
engine = create_engine(f'mysql://{conn_str}')

#check for tables in database
#engine.table_names()

#enter data into database
***df***.to_sql(name="airbnb_data_tbl", con=engine, if_exists="append",index=False)

In [6]:
# from opencage.geocoder import OpenCageGeocode

ImportError: No module named opencage.geocoder

In [8]:
API_key = "30c5a732512c41af8f052a21f806fcce"

geocoder = OpenCageGeocode(API_key)

city_country = 'Greater-manchester, United kingdom'

results = geocoder.geocode(city_country)

print (results)

[{u'confidence': 2, u'geometry': {'lat': 53.5065248, 'lng': -2.3374401}, u'formatted': u'Greater Manchester, England, United Kingdom', u'bounds': {u'northeast': {'lat': 53.6857188, 'lng': -1.9096224}, u'southwest': {'lat': 53.3273147, 'lng': -2.7305207}}, u'components': {u'political_union': u'European Union', u'_type': u'county', u'ISO_3166-1_alpha-3': u'GBR', u'country': u'United Kingdom', u'ISO_3166-1_alpha-2': u'GB', u'county': u'Greater Manchester', u'state': u'England', u'state_district': u'North West England', u'country_code': u'gb', u'state_code': u'ENG', u'continent': u'Europe'}, u'annotations': {u'MGRS': u'30UWE4394228823', u'roadinfo': {u'drive_on': u'left', u'speed_in': u'mph'}, u'callingcode': 44, u'OSGB': {u'gridref': u'SD 777 011', u'easting': 377716.905, u'northing': 401144.045}, u'sun': {u'rise': {u'civil': 1560998820, u'astronomical': 0, u'apparent': 1561002000, u'nautical': 1560993000}, u'set': {u'civil': 1561066500, u'astronomical': 0, u'apparent': 1561063320, u'naut

In [5]:
lat = results[0]['geometry']['lat']
lng = results[0]['geometry']['lng']
print (lat, lng)

(41.8755616, -87.6244212)


In [ ]:
list_lat = []   # create empty lists
list_long = []

for index, row in df_crime_more_cities.iterrows(): # iterate over rows in dataframe

    City = row['City']
    State = row['State']       
    query = str(City)+", "+str(State)
                  
    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    long = results[0]['geometry']['lng']

    
    list_lat.append(lat)
    list_long.append(long)
    
# create new columns from lists    
df_crime_more_cities['lat'] = list_lat   
df_crime_more_cities['lon'] = list_long

## Lat/Long Calculation Examples

In [ ]:
from math import radians, cos, sin, asin, sqrt 

In [ ]:
# Python 3 program to calculate Distance Between Two Points on Earth 

def distance(lat1, lat2, lon1, lon2): 
      
    # The math module contains a function named 
    # radians which converts from degrees to radians. 
    lat1 = radians(lat1) 
    lat2 = radians(lat2)
    lon1 = radians(lon1) 
    lon2 = radians(lon2) 
 
       
    # Haversine formula  
    dlon = lon2 - lon1  
    dlat = lat2 - lat1 
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles 

    r = 6371 # or 3958.756 for miles.
       
    # calculate the result 
    return(c * r) 
      
      
# driver code  
lat1 = 53.32055555555556
lat2 = 53.31861111111111
lon1 = -1.7297222222222221
lon2 =  -1.6997222222222223
print(distance(lat1, lat2, lon1, lon2), "K.M") 

In [ ]:
Output:
2.0043678382716137 K.M

## ANOTHER EXAMPLE

In [ ]:
from math import radians, sin, cos, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))